In [40]:
import pandas as pd 

target_df = pd.read_csv('/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/atrophy_seeds_2023/metadata/atrophy_roi_scores/baseline_atrophy_scores_pivoted.csv')
target_df

,Unnamed: 0,subid,Age,Sex,Cohort,Diagnosis,Region,Tissue,Visual,Measurement
0,0,002_S_0816,70.838356,F,1.0,Alzheimer,Cerebellum,CSF,0.0,-25421.113250
1,1,002_S_0816,70.838356,F,1.0,Alzheimer,Cerebellum,GM,0.0,-5411.426766
2,2,002_S_0816,70.838356,F,1.0,Alzheimer,Cerebellum,WM,0.0,-19072.674790
3,3,002_S_0816,70.838356,F,1.0,Alzheimer,Cerebellum,CTh,0.0,0.000000
4,4,002_S_4270,NaN,F,1.0,Normal,Cerebellum,CSF,0.0,-9935.777020
...,...,...,...,...,...,...,...,...,...,...
36787,36787,941_S_4420,NaN,NaN,NaN,NaN,Temporal,CTh,NaN,NaN
36788,36788,941_S_4764,NaN,NaN,NaN,NaN,Temporal,CSF,NaN,-37860.577210
36789,36789,941_S_4764,NaN,NaN,NaN,NaN,Temporal,GM,NaN,9908.644310
36790,36790,941_S_4764,NaN,NaN,NaN,NaN,Temporal,WM,NaN,2249.815484


In [41]:
import numpy as np
target_df.drop_duplicates(subset=['subid'], inplace=True)
target_df.dropna(subset=['Diagnosis'], inplace=True)
target_df.drop(target_df[target_df['Cohort']==0].index, inplace=True)

subids = target_df['subid'].unique()
diagnoses = target_df['Diagnosis'].unique()

counter = {}

for diagnosis in diagnoses:
    counter[diagnosis]= []
for sub in subids:
    dx = target_df.loc[target_df['subid']==sub, 'Diagnosis'].values[0]
    if dx is not np.nan: counter[dx].append(sub)

for k, v in counter.items():
    print(f'Numer of patients with {k}: {len(v)}')

In [34]:
import pandas as pd

# Function to perform the pseudo-pivot on the dataset
def pseudo_pivot_with_surface_values(df, regions, tissue_types, duplicate_columns):
    pivoted_rows = []

    # Ensure 'Surface_' values are assigned to 'CTh' in the pivoted data and preserved after cohort matching
    for region in regions:
        visual_col = f'Visual_{region}'  # Visual outcome column
        for _, row in df.iterrows():
            for tissue in tissue_types:
                new_row = {col: row[col] for col in duplicate_columns}  # Duplicate demographic columns
                new_row['Region'] = region  # Add brain region
                new_row['Tissue'] = tissue  # Add tissue type
                new_row['Visual'] = row[visual_col]  # Add visual outcome
                measurement_col = f'{tissue}_{region}'  # Construct the measurement column name for CSF, GM, WM

                # Assign the measurement value, preserving Surface_ values if they exist
                new_row['Measurement'] = row[measurement_col] if measurement_col in df.columns else None
                pivoted_rows.append(new_row)

    # Convert the list of dictionaries to a new DataFrame
    return pd.DataFrame(pivoted_rows)

# Usage example with relevant input variables:
regions = ['Cerebellum', 'Subcortex', 'MTL', 'Occipital', 'Frontal', 'Parietal', 'Temporal']
tissue_types = ['CSF', 'GM', 'WM', 'CTh']
duplicate_columns = ['subid', 'Age', 'Sex', 'Cohort', 'Diagnosis']

# Assuming df is the DataFrame with Surface_ values already populated for cohort 1.
pivoted_df = pseudo_pivot_with_surface_values(target_df, regions, tissue_types, duplicate_columns)


In [35]:
pivoted_df.to_csv('/Volumes/Expansion/datasets/adni/metadata/cleaned_information/baseline_atrophy_scores_pivoted.csv')